## Question 3

Design a feature selection algorithm to find the best features for classifying the Mnist dataset. Implement a bidirectional search algorithm using the provided objective function as the measure for your search algorithm.

Use the first 10000 samples of training set in the Mnist dataset for feature selection and training set for kNN approach. Use Euclidean distance to calculate Inter-class.

The objective function should be based on this equestion:

### J = Inter Class distance

In [294]:
import gzip
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean

image_size = 28
training_samples = 60000

# Importing Train Data
f_train = gzip.open('train-images-idx3-ubyte.gz','r')
f_train.read(16)
buf = f_train.read(image_size * image_size * training_samples)
train_data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
# train_data = data.reshape(num_images, image_size, image_size)
train_data = train_data.reshape(training_samples, image_size* image_size)


# Importing Train Labels
f_train_label = gzip.open('train-labels-idx1-ubyte.gz','r')
f_train_label.read(8)
buf = f_train_label.read(training_samples)
train_labels = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)


# Importing Test Data
testing_images = 10000
f_test = gzip.open('t10k-images-idx3-ubyte.gz','r')
f_test.read(16)
buf = f_test.read(image_size * image_size * testing_images)
test_data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
test_data = test_data.reshape(testing_images, image_size * image_size)

# Importing Test Labels
f_test_label = gzip.open('t10k-labels-idx1-ubyte.gz','r')
f_test_label.read(8)
buf = f_test_label.read(testing_images)
test_labels = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)


train_data = train_data[0:10000]
train_labels = train_labels[0:10000]

In [302]:
from sklearn.neighbors import KNeighborsClassifier
def apply_knn(train_data, train_labels, k):
    
    train_d =train_data[:8000]
    train_l =train_labels[:8000]
    
    test_d = train_data[8001:]
    test_l =train_labels[8001:]
    
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(train_d,train_l)
    
    correct = 0
    for i in range(len(test_d)):
        if model.predict([test_d[i]]) == test_l[i]: correct+=1
    
    return correct/len(test_d)

a) Select the set of {10, 50, 150, 392} features based on the implemented feature selection approach and report the accuracy on the test set of MNIST based on kNN with k = 3. Note: you can take advantage of data structure tricks to speed up the efficiency of kNN algorithm.

In [296]:
my_train_data = train_data
new_train_data = None

for _ in range(392):
 
    # SFS
    main_mean = np.mean(my_train_data, axis=0)
    mean_i = []
    for i in np.unique(train_labels):
        indices_of_i = np.where(train_labels == i)[0]
        c_i = np.take(my_train_data, indices_of_i, axis=0)
        mean_i.append(np.mean(c_i, axis=0))    

    total_no_of_features= len(mean_i[0])
    J = np.zeros(total_no_of_features)

    for i in range(total_no_of_features):
        for mean in mean_i:
            distance = euclidean(main_mean[i],mean[i])
            J[i]+= distance

    if new_train_data is None:
        new_train_data=my_train_data[:,np.argmax(J)]
    else:  
        new_train_data = np.c_[new_train_data, my_train_data[:,np.argmax(J)]]

    my_train_data = np.delete(my_train_data,np.argmax(J), 1)

    # SBS
    main_mean = np.mean(my_train_data, axis=0)
    mean_i = []
    for i in np.unique(train_labels):
        indices_of_i = np.where(train_labels == i)[0]
        c_i = np.take(my_train_data, indices_of_i, axis=0)
        mean_i.append(np.mean(c_i, axis=0))    

    total_no_of_features= len(mean_i[0])
    J = np.zeros(total_no_of_features)

    for i in range(total_no_of_features):
        for mean in mean_i:
            distance = euclidean(main_mean[i],mean[i])
            J[i]+= distance
            
    my_train_data = np.delete(my_train_data,np.argmin(J), 1)

print(new_train_data.shape)

(10000, 392)


In [ ]:
new_train_data_10 = new_train_data[:,:10]
new_train_data_50 = new_train_data[:,:50]
new_train_data_150 = new_train_data[:,:150]
new_train_data_392 = new_train_data

accuracies = []
accuracies.append(apply_knn(new_train_data_10,train_labels, 3))
accuracies.append(apply_knn(new_train_data_50,train_labels, 3))
accuracies.append(apply_knn(new_train_data_150,train_labels, 3))
accuracies.append(apply_knn(new_train_data_392,train_labels, 3))
print(accuracies)

b) Visualize the selected features for each set in {10, 50, 150, 392} by a zero 2-D plane where the selected features are pixels set to a value of 1. Compare the 4 different planes.

c) Apply LDA on the dataset and report the accuracy based on kNN with k =3. Compare the achieved accuracy by the reported accuracies in part (a). Note: you need to implement LDA method by yourself.

In [223]:
print(train_data.shape)
print(train_labels.shape)



(10000, 784)
(10000,)


In [222]:
apply_knn(train_data,train_labels)

0.9349674837418709
